In [ ]:
''' 
Code to transform CERRA grip files into raster files and select aand save a specified area

The Copernicus European Regional ReAnalysis (CERRA) products include a wide variety of different 
meteorological reanalysis data with a temporal resolution of 3 hours.

In comparison to other reanalysis products, CERRA only covers the European continent, but with an increases spatial resolution
of 5.5 km.

More information can be found 
a) on the distinct CERRA product pages:
https://cds.climate.copernicus.eu/cdsapp#!/search?type=dataset&text=CERRA
b) the Copernicus user guide:
https://confluence.ecmwf.int/display/CKB/Copernicus+European+Regional+ReAnalysis+%28CERRA%29%3A+product+user+guide

'''

In [1]:
import re
import pandas as pd
import geopandas as gpd
import numpy as np
from osgeo import osr, ogr, gdal
from datetime import datetime, timedelta

In [6]:
'''
function to get the selected band of the grib file based on the 

input:
grib_file: opened grip file
met_var: string with the meteorological variable that is to be extracted 

output:
matching_bands: band number of the grib file that match the meteorological variable
'''

def get_bands_based_on_grib_element(grib_file, met_var):

    matching_bands = []

    # Loop through each band and check the GRIB_ELEMENT metadata
    for i in range(1, grib_file.RasterCount + 1):  # GDAL band indices start at 1
        band = grib_file.GetRasterBand(i)
        metadata = band.GetMetadata()
        
        if 'GRIB_ELEMENT' in metadata and metadata['GRIB_ELEMENT'] == met_var:
            matching_bands.append(i)

    return matching_bands

In [12]:
'''
function to reproject the CERRA data into a new crs

input:
cerra_data: the opened grib-file with. Its projection has to be set
list_of_bands_to_warp: a list with band numbers that should be warped into the new crs.
                       Band numbers can be found with "get_bands_based_on_grib_element"
target_crs: the target crs

output:
output_vrt_name: name of the in-memory .tif-file
output_dataset: the actual in-memory .tif-file
'''

def warp_CERRA_to_targetCRS(cerra_data, list_of_bands_to_warp, target_crs):
    
    # Extract the desired bands and save the data in an in-memory dataset
    in_memory_name = "/vsimem/temp_band.vrt"
    gdal.Translate(in_memory_name, cerra_data, format="VRT", bandList=list_of_bands_to_warp)
    
    # open the in-memory dataset
    temp_dataset = gdal.Open(in_memory_name)

    # Warp the in-memory dataset to an in-memory .tif-file with the target CRS 
    output_vrt_name = "/vsimem/output_in_memory.tif"
    gdal.Warp(output_vrt_name, temp_dataset, resampleAlg="bilinear", dstSRS=target_crs)

    # open the new in-memory .tif 
    output_dataset = gdal.Open(output_vrt_name)
    
    # At this point, the data in 'output_dataset' is the warped result, kept in memory.
    # unlink the first dataset
    gdal.Unlink(in_memory_name)
    
    # return the name of the in-memory .tif-file, as well as the file itself
    return output_vrt_name, output_dataset

In [13]:
'''
Function to get raster values of specific points

input:
ds: grib-file data, transformed into the virtual .tif-file and reprojected into the correct crs 
    This is the result of the 'warp_CERRA_to_targetCRS' function
gt: geotransform of the grib-file data, transformed into the virtual .tif-file and reprojected into the correct crs
x: x/lon values of the point
y: y/lat-values of the point

ouput:
values: values of the meteorological variable at the specific points
time: the initial grib-file can include the meteorological data of multiple days
      to link the resulting values to the correct date, 
      the date of every value point is saved in this variable 
'''

def get_raster_value_at_point(ds, gt, x, y):
    col = int((x - gt[0]) / gt[1])
    row = int((y - gt[3]) / gt[5])
    
    # Bounds check
    if (col < 0 or col >= ds.RasterXSize or 
        row < 0 or row >= ds.RasterYSize):
        print(f"Warning: Point ({x}, {y}) is out of raster bounds.")
        return None
    

    values = []
    time = []
    
    for timestamp in range(1, (ds.RasterCount + 1)):
        
        band = ds.GetRasterBand(timestamp)  # Assuming you want the first band
        metadata = band.GetMetadata()
        time.extend([pd.to_datetime(re.search(' REF_TIME=(.*)Z', metadata["GRIB_IDS"]).group(1))])
        
        # Check band validity
        if not band:
            print("Warning: Band is not valid.")
            return None
        
        data = band.ReadAsArray(col, row, 1, 1)
        
        # Check if data is None
        if data is None:
            print(f"Warning: No data for point ({x}, {y}).")
            return None
        
        values.extend([data[0][0]])
    
    return values, time

In [21]:
'''
combines all functions into a single function, able to 
a) retrieve the bands of investigated variables,
b) reproject the CERRA data into a new CRS, and,
c) extract meteorological values for specific points 

input: 
shp_with_points: shapefile including the point coordinates 
cerra_grib: the grip-file, openend in python (gdal.Open(path))
meteo_index: string of the name of the meteorological variable 

output:
values_all_points: list with all values of the meteorological variable at the point positions
coord_points: point coordinates 
time: date references of the extracted values
'''

def get_point_values_of_CERRA_grib(shp_with_points, cerra_grib, meteo_index):
    
    # get the layer of the shapefle that includes the geometry of the points
    layer = shp_with_points.GetLayer()
    # get the crs of the shapefile. This is the target crs of the CERRA data 
    # and ensures that both datasets are in the same crs
    target_crs = layer.GetSpatialRef()
    
    # create a list with band numbers that belong to the meteorological variables 
    bands_with_wdir = get_bands_based_on_grib_element(cerra_grib, meteo_index)
    # reproject the grib-file into the wanted crs
    vrt_dataset, output_dataset = warp_CERRA_to_targetCRS(cerra_grib,bands_with_wdir,target_crs)
    
    # get bounds of raster for the point extraction
    raster_gt = output_dataset.GetGeoTransform()
    raster_proj = osr.SpatialReference()
    raster_proj.ImportFromWkt(output_dataset.GetProjection())
    
    # Iterate through the shapefile and extract the values for every point in the shapefile and every band of the grib file
    values_all_points = []
    coord_points = []
    time_all_points = []

    for feature in layer:
        # select the specific point for which to extract the point values
        geom = feature.GetGeometryRef()
        x = geom.GetX()
        y = geom.GetY()
        # use function to get the values of the specific point for every band
        values, time = get_raster_value_at_point(output_dataset, raster_gt, x, y)
        # append the list with the values of one point. 
        # Afterwards, this variable includes the point values of every point in an individual list
        values_all_points.append(values)
        coord_points.append([x,y])
        time_all_points.append(time)
        
    # Cleanup datasets
    gdal.Unlink(vrt_dataset)
    
    # return three lists: the meteorological data at the point positions,
    # the point coordinates, and the date 
    return values_all_points, coord_points, time

### Example: Extract wind direction for specific point loactions from a CERRA .grib-file

In [2]:
# set the grid information for the CERRA-data
# This information was obtained from the user guide:
# https://confluence.ecmwf.int/display/CKB/Copernicus+European+Regional+ReAnalysis+%28CERRA%29%3A+product+user+guide

source_crs = osr.SpatialReference()
source_crs.SetProjCS("Lambert Conformal Conic")
source_crs.SetLCC(50, 50, 50, 8, 0, 0)
source_crs.SetGeogCS("WGS 84 based on Sphere", "WGS_84", "Sphere", 6371229, 0.0)

# set the target crs for the CERRA-data to the EPSG 4326
target_crs = osr.SpatialReference()
target_crs.ImportFromEPSG(4326)

0

In [36]:
# Open the GRIB file
grib_dataset = gdal.Open("/.grib")
# Set the projection (assigning the CRS to the GRIB file)
grib_dataset.SetProjection(source_crs.ExportToWkt())

0

In [ ]:
# open a shapefile with the coordinates of the points for which the weather data should be extracted
shapefile_path = ".../points_to_extract_from_CERRA.shp"
gdf_shapefile = ogr.Open(shapefile_path)

In [37]:
# get the wind direction ("WDIR") for all the specified points 
wind_direction, coordinates, time = get_point_values_of_CERRA_grib(gdf_shapefile,grib_dataset,"WDIR",target_crs)